#### Perfect Prediction Investigating
* most likely data leakage

**Observations**:
* Showing perfect scores across the board, which is not realistic. Human behavior is unpredicatable, so perfect prediction is not likely. I was aiming for above 0.7.
* **Data Leakage** - most likely cause. Some informatin abot the target variable `Churn  has inadvertently "leaked" into my feature set (X_train and X_test). I don't think the model is learning to predict, I think it has the answers.
* **Investigating Leakage**:
    * Is there a feature in df_model_ready that is providing the Churn answer?
        * ***MOST DIRECT CHECK*** is to look at the correlation between features and the Churn target variable.
        * If any feature has a correlation of +1 or -1 (or extremely close to it) with Churn, that's a very strong sign of leakage or that the feature is a direct proxy for the target.  

* See notebook day-6-7-data_leakage.ipynb


In [ ]:
import pandas as pd
import numpy as np

# Load the feature-engineered DataFrame
df_for_modeling_corr_check = pd.read_parquet(r"C:\Users\comat\GitProjects\customer-churn-ai\data\telco_churn_feature_engineered.parquet")
# Make sure this DataFrame has the 'Churn' column and all your engineered features.

print("Calculating correlations with the 'Churn' column...")
# Ensure 'Churn' is numeric (it should be 0 or 1 from our previous steps)
if 'Churn' in df_for_modeling_corr_check.columns and pd.api.types.is_numeric_dtype(df_for_modeling_corr_check['Churn']):
    correlation_with_churn = df_for_modeling_corr_check.corr()['Churn'].sort_values(ascending=False)
    print("\nTop correlations with Churn:")
    print(correlation_with_churn.head(10)) # Show top positive correlations
    print("\nBottom correlations with Churn (most negative):")
    print(correlation_with_churn.tail(10)) # Show top negative correlations
else:
    print("Could not calculate correlations. Ensure 'df_model_ready' is loaded and 'Churn' is a numeric column.")

Calculating correlations with the 'Churn' column...

Top correlations with Churn:
Churn                             1.000000
HF_neg                            0.988827
HF_nue                            0.599272
InternetService_Fiber optic       0.308020
PaymentMethod_Electronic check    0.301919
MonthlyCharges                    0.193356
PaperlessBilling_Yes              0.191825
SeniorCitizen                     0.150889
HF_Label_Neutral                  0.137827
StreamingTV_Yes                   0.063228
Name: Churn, dtype: float64

Bottom correlations with Churn (most negative):
Dependents_Yes           -0.164221
TechSupport_Yes          -0.164674
OnlineSecurity_Yes       -0.171226
Contract_One year        -0.177820
TotalCharges             -0.199428
TenureGrp_61-72 Months   -0.225500
InternetService_No       -0.227890
Contract_Two year        -0.302253
HF_pos                   -0.996824
HF_Label_Positive        -0.998908
Name: Churn, dtype: float64


---
**Investigation Assessment:**  
  
  
Clearly the following features are too close to 1 or -1:  
HF_neg:             0.988827  
HF_pos:            -0.996824  
HF_Label_Positive: -0.998908  

* This means the Hugging Face sentiment analysis was almost perfect.
* In reality, this is because I created the Customer Reviews to be dependent on Churn or No Churn, unknowingly, baking in data leakage.
* In real life, if this occurs, it's a sign that the CustomerReview column likely contains text that was written at the point of churn, after churn has happened, or expresses a very clear and immediate intent to churn.  

- If the features used for training know the outcome because the Customer Review Text states it, the model isn't learning.
- In order to build a model that can predict which customers are likely to churn in the future, based on signals before they've firmly decided or explicitly stated they are leaving, I have to correct the leakage

---
***Plan***:
1. Return to df_model_ready at the beginning of day-5-split-scale.ipynb, before it's used to split into X and y. It has all the engineered features from day-5-feature-engineering.
2. Remove HF_neg, HF_nue, HF_pos, and all one-hot encoded columns derived from HF_Label
3. Re-split the Data
4. Re-scale Features
5. Retrain and Re-evaluate
